## (Rass Second Movement) Third Movement: Periodic Defender with CVE Data

In [1]:
print("""This Cell is necessary or else my IDE will start running cells simultaneously which
      then leads to issues with unfinished imports""")

This Cell is necessary or else my IDE will start running cells simultaneously which
      then leads to issues with unfinished imports


In [2]:
# Import required libraries
import os
import logging
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt

## New Stuff

In [3]:
# Check if variables exist, otherwise default to False
image_mode = globals().get('image_mode', False)
debug_mode = globals().get('debug_mode', False)
RUN_BASELINE_ONLY = globals().get('RUN_BASELINE_ONLY', False)

# If you are NOT using "extrapolate.ipynb" uncomment any you want to set to true
# however remember to re-comment them out later because otherwise extrapolate will be ignored
#image_mode = True
#debug_mode = True
#RUN_BASELINE_ONLY = True

print(f"Current settings: image_mode={image_mode}, debug_mode={debug_mode}, RUN_BASELINE_ONLY={RUN_BASELINE_ONLY}")

Current settings: image_mode=False, debug_mode=False, RUN_BASELINE_ONLY=False


In [4]:
experiment_name = "experiment_3"

In [5]:
# path construct
log_path = os.path.join(os.getcwd(), str(experiment_name) + '.log')

if os.path.exists(log_path):
    os.remove(log_path)

logger = logging.getLogger()
handler = logging.FileHandler(log_path, mode='w')
handler.setFormatter(logging.Formatter('%(message)s'))
logger.addHandler(handler)
logger.setLevel(logging.INFO)

# concat of name/date
logger.info('[1] "' + str(experiment_name) + '.log"')
logger.info('[1] "' + datetime.now().strftime("%a %b %d %H:%M:%S %Y") + '"')

# Set up subgraph logger 
if not RUN_BASELINE_ONLY: 
    subgraph_log_path = os.path.join(os.getcwd(), 'sub_' + str(experiment_name) + '.log')
    
    if os.path.exists(subgraph_log_path):
        os.remove(subgraph_log_path)
        
    subgraph_handler = logging.FileHandler(subgraph_log_path, mode='w')
    subgraph_handler.setFormatter(logging.Formatter('%(message)s'))
    

# Function to switch between loggers
def switch_logger(use_subgraph_logger=False):
    
    # remove handlers
    current_handlers = logger.handlers[:]
    i = 0
    while i < len(current_handlers):
        logger.removeHandler(current_handlers[i]) 
        i = i + 1
        
    if use_subgraph_logger:
        logger.addHandler(subgraph_handler)
        logger.info('[1] "sub_' + str(experiment_name) + '.log"')
        logger.info('[1] "' + datetime.now().strftime("%a %b %d %H:%M:%S %Y") + '"')
    else:
        logger.addHandler(handler)

In [6]:
# Global configuration
# This is the default weight value we will insert any time we we 
# need to insert a weight value for a trivial edge in the graph
# Why 0 ?  Because we have "hardness = exp(-w)" so we get hardness = 1 for w = 0
# hardness = 1 means that edge is trivial to traverse
DEFAULT_WEIGHT_VALUE = 0  

In [7]:
%run attack_graph_MIR100.ipynb

### Create Subgraphs

In [8]:
#drop_pct = 0.75

In [9]:
%run create_subgraphs.ipynb

### Movement Function

In [10]:
attack_rate_list = [0]   
defense_rate_list = [0]  

In [11]:
def random_steps(route, attack_rate=None, defense_rate=None, graph=None):
    
    # Part01: extract hardness values from all edges & append them
    hardness = []
    for i in range(len(route) - 1):
        start_node = route[i]
        end_node = route[i + 1]
        
        # Initialize vars for max weight loop
        weights = []
        # Collect weights for max
        for edge in graph[start_node][end_node].values():
            weights.append(edge.get('weight', DEFAULT_WEIGHT_VALUE))
        # Get maximum weight
        max_weight = max(weights) if weights else DEFAULT_WEIGHT_VALUE
        
        min_weights = []
        # Collect weights for min
        for edge in graph[start_node][end_node].values():
            min_weights.append(edge.get('weight', DEFAULT_WEIGHT_VALUE))
        # Get minimum weight
        min_weight = min(min_weights) if min_weights else DEFAULT_WEIGHT_VALUE
        
        hardness.append(np.exp(-min_weight))
       
    hardness = np.array(hardness)

    # print(f'Hardness: {hardness}')
    
    # Part02: Based on extracted hardness values

    cp = np.concatenate(([1.0], np.cumprod(hardness))) #cumulative probs

    stop_probs = np.concatenate((1 - hardness, [1.0]))

    # part03: generate final distribution
    
    pdf = cp * stop_probs

    sp = pdf.sum()
    # case where prob are essentially zero
    if sp < 1e-15:
        pdf = np.full_like(pdf, 1e-7)

    # Normalize 
    ret = pdf/ sp
    return ret

In [12]:
# %run ctr-core_simple.ipynb
%run exp_ctr-core.ipynb
main()

We start with the baseline graph calculation!
Now we are going to run the subgraph analysis
The subgraph analysis is done!


In [13]:
with open(experiment_name+'.log', 'r') as f:
    print(f.read())

[1] "experiment_3.log"
[1] "Sun Nov 30 15:03:11 2025"



BASELINE RUN: BOTH ATTACKER AND DEFENDER HAVE FULL GRAPH KNOWLEDGE



Debug - Current Attack paths:
  0: [0, 1, 5, 15, 'c(12,13,14,16)']
  1: [0, 3, 6, 8, 'c(12,13,14,16)']
  2: [0, 3, 6, 8, 10, 15, 'c(12,13,14,16)']
  3: [0, 3, 8, 'c(12,13,14,16)']
  4: [0, 3, 8, 10, 15, 'c(12,13,14,16)']
  5: [0, 4, 7, 'c(12,13,14,16)']
  6: [0, 4, 7, 10, 15, 'c(12,13,14,16)']
  7: [0, 2, 'c(12,13,14,16)']
  8: [0, 2, 9, 'c(12,13,14,16)']
  9: [0, 2, 10, 15, 'c(12,13,14,16)']
  10: [0, 2, 11, 'c(12,13,14,16)']

+++++++++++++++++++++++++++++

The virtual target nodeid is c(12,13,14,16)

attack rate =  0 , defense rate =  0 

	equilibrium for multiobjective security game (MOSG)

optimal defense strategy:
         prob.
1 0.000000e+00
10 0.000000e+00
11 2.720264e-01
15 1.852234e-01
2 0.000000e+00
3 0.000000e+00
4 0.000000e+00
5 0.000000e+00
6 0.000000e+00
7 0.000000e+00
8 5.427503e-01
9 0.000000e+00

worst case attack strategies per goal:
        